# 🧠 Обработка JSONL-файлов через LLaMA API

In [16]:
from google.colab import files
import zipfile
import os
from pathlib import Path

# Загрузка архива
uploaded = files.upload()
zip_path = list(uploaded.keys())[0]

# Папка для распаковки
extract_dir = "content/unzipped"
Path(extract_dir).mkdir(parents=True, exist_ok=True)

# Распаковка архива
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_dir)
    print("📦 Архив распакован")

Saving drive-download-20250722T115256Z-1-001.zip to drive-download-20250722T115256Z-1-001 (1).zip
📦 Архив распакован


In [17]:
from pathlib import Path
import json

texts = []

# Ищем все jsonl-файлы и загружаем тексты
for file in Path(extract_dir).glob("*.jsonl"):
    with open(file, encoding="utf-8") as f:
        for line in f:
            try:
                obj = json.loads(line)
                if "text" in obj:
                    texts.append(obj["text"])
                elif "abstract" in obj:
                    texts.append(obj["abstract"])
            except:
                pass

print(f"Загружено текстов: {len(texts)}")

Загружено текстов: 500


In [21]:
import requests
import json
import time

# Настройки подключения
API_URL = "http://80.209.242.40:8000/v1/chat/completions"
HEADERS = {"Content-Type": "application/json"}

# System prompt
system_prompt = '''
Ты — мультиагентная LLM-система нового поколения, участвующая в научно-инженерном хакатоне AI x Longevity.

Твоя задача — действовать как один агент с несколькими режимами. Каждый режим отвечает за определённый тип задачи.
Ты автоматически определяешь нужный режим по содержанию сообщения, если пользователь явно не указал его.
Внешние инструменты: PubMed API, Neo4j, FAISS и т.п.

Если пользователь:
- задаёт вопрос о механизме или концепции — активируй режим /объяснение;
- предлагает гипотезу или просит придумать её — активируй /гипотеза;
- даёт список задач и просит выбрать — активируй /приоритизация;
- просит извлечь данные из текста — активируй /извлечение;
- просит связать гены, белки, болезни — активируй /граф;
- просит резюме за день — активируй /daily-report.

Если информации недостаточно — кратко запроси уточнение. Не придумывай критичные элементы сам.

Избегай вступлений и воды. Пиши строго по делу. Не используй фразы вроде «как ИИ...». Переходи сразу к выполнению задачи.

Обращайся к пользователю как к исследователю или инженеру. Тон — деловой, чёткий, научный.

# Доступные режимы:

1. /извлечение — извлечение сущностей, гипотез, связей, классификация, достоверность.
2. /приоритизация — ранжирование задач или гипотез по новизне, зрелости, влиянию, доказанности.
3. /объяснение — объяснение, почему задача или тема приоритетна, с логической цепочкой.
4. /граф — построение словесного графа знаний (гены, белки, вмешательства, механизмы).
5. /гипотеза — генерация научной гипотезы и плана проверки.
6. /daily-report — краткий отчёт по приоритетной задаче дня.

# Форматируй ответы так:
- Структурируй ответ по пунктам.
- Пиши краткими абзацами, без лишней воды.
- В конце ответа укажи краткий вывод: «Вывод: ...»
- При наличии источников — укажи их (реальные или помеченные как гипотетические).

# Пример вызова режима:

/гипотеза
Сгенерируй научную гипотезу по FOXO3. Приведи:
- факты-основания;
- формулировку;
- план тестирования;
- возможные риски.

# Дополнительно:

- Если пользователь говорит не по теме — мягко переформулируй или предложи режим.
- Не повторяй инструкции в ответах.
- Всегда придерживайся контекста: биология старения, клеточные и молекулярные механизмы, гены, интервенции, эпигенетика, preclinical и клинические исследования.
"""
'''

# Пример вызова
data = {
    "model": "meta-llama/Llama-3-8b-instruct",  # Укажи нужную модель, если другая
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "/гипотеза\nСгенерируй гипотезу по роли SCFAs в когнитивном старении"}
    ]
}

# Отправка запроса
response = requests.post(API_URL, headers=HEADERS, data=json.dumps(data))

# Обработка ответа
output = response.json()["choices"][0]["message"]["content"]
print(output)

**Факты-основания:**
- SCFAs (короткоцепочечные жирные кислоты) производятся кишечными микроорганизмами в процессе ферментации неперевариваемых углеводов.
- SCFAs, такие как ацетат, пропионат и бутират, играют ключевую роль в поддержании здоровья кишечника и влияют на различные физиологические процессы, включая метаболизм, иммунный ответ и функцию мозга.
- Нарушения баланса кишечной микробиоты связаны с когнитивным снижением и старением.

**Формулировка гипотезы:**
SCFAs, производимые кишечной микробиотой, могут замедлить когнитивное старение, модулируя воспаление, окислительный стресс и эпигенетические изменения в мозге, что в конечном итоге приводит к улучшению когнитивных функций у пожилых людей.

**План тестирования:**
1. **Эксперименты in vitro:** Исследовать влияние SCFAs на воспаление и окислительный стресс в клетках мозга.
2. **Исследования на животных моделях:** Провести эксперименты на старых мышах, чтобы оценить эффекты диеты, обогащенной SCFAs, на когнитивные функции и изме

In [33]:
import pandas as pd

# Загружаем входной CSV (замени имя файла при необходимости)
df = pd.read_csv("llama_input.csv")  # если файл называется иначе — замени название
texts = df["input"].tolist()

results = []

In [37]:
import csv

prompts = [
    "/гипотеза Сгенерируй гипотезу по роли FOXO3 в старении",
    "/объяснение Почему митохондриальные мутации ускоряют старение?",
    "/приоритизация Отсортируй задачи: омоложение кожи, восстановление мышц, удаление сенесцентных клеток"
]

with open("llama_input.csv", "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["input"])
    for prompt in prompts:
        writer.writerow([prompt])

In [38]:
for text in texts:
    data = {
        "model": "meta-llama/Llama-3-8b-instruct",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": text}
        ]
    }

    try:
        response = requests.post(API_URL, headers=HEADERS, data=json.dumps(data))
        output = response.json()["choices"][0]["message"]["content"]
    except Exception as e:
        output = f"Ошибка: {str(e)}"

    results.append({"input": text, "output": output})
    time.sleep(1)  # Пауза между запросами, чтобы не перегрузить сервер

In [40]:
df = pd.read_csv("llama_input.csv")
texts = df["input"].tolist()

results = []

In [41]:
for text in texts:
    data = {
        "model": "meta-llama/Llama-3-8b-instruct",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": text}
        ]
    }

    try:
        response = requests.post(API_URL, headers=HEADERS, data=json.dumps(data))
        output = response.json()["choices"][0]["message"]["content"]
    except Exception as e:
        output = f"Ошибка: {str(e)}"

    results.append({"input": text, "output": output})
    time.sleep(1)  # Пауза между запросами, чтобы не перегрузить сервер

In [42]:
import csv

with open("llama_output.csv", "w", encoding="utf-8", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["input", "output"])
    writer.writeheader()
    writer.writerows(results)

In [43]:
from google.colab import files
files.download("llama_output.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>